In [1]:
import pandas as pd
import gzip
import re

In [2]:
random = pd.read_pickle("random.pkl")
news = pd.read_pickle("news.pkl")

In [45]:
print(f"There are {len(random[random.str.contains('banned from')])} occurences in the random sample and {len(news[news.str.contains('banned from')])} in the news sample")

There are 1637 occurences in the random sample and 1604 in the news sample


In [3]:
corpus = pd.concat([random, news])

## 1rst iteration using a seed pattern. retreiving the entity/platform pairs

In [4]:
base_entity = "(\S+)(?:'m| am| was| is| are| were|) banned from"
base_platform = "banned from(?: the)? (\S+)"

In [28]:
banned = []
for l in corpus[corpus.str.contains('banned from')]:
    ent = re.search(base_entity, l)
    plat = re.search(base_platform, l)
    if ent and plat:
        banned.append([ent.group(1), plat.group(1)])
    else:
        pass

In [60]:
pronouns = ['i','he', 'she', 'they', 'you', 'me', 'her', 'them']
print(f"Number of entity being 'i' or 'he', 'she', 'they', 'you': {len([a for a in banned if a[0] in pronouns])} \n")
print(f"Out of {len(banned)}")

Number of entity being 'i' or 'he', 'she', 'they', 'you': 317 

Out of 3040


In [26]:
base_pairs = pd.DataFrame(banned, columns = ['entity', 'platform'])

In [27]:
base_pairs.groupby('entity').count().sort_values(by='platform', ascending = False).head(5)

,platform
entity,
be,259
i,223
been,158
got,145
get,76


## Second iteration

In [37]:
base_pairs

,entity,platform
0,i,r/2xchromosomes
1,team,"olympics,"
2,temporarily,playing
3,me,news
4,get,this
...,...,...
3035,she,leaving
3036,woman,walmart
3037,millennials',disney
3038,got,google


Removing all pairs for which the entity was a pronoun (317 pairs)

In [62]:
base_pairs_cleaned = base_pairs[base_pairs.apply(lambda x: x['entity'] not in pronouns, axis = 1)]

In [ ]:
res = []
condition = lambda x, y: base_pairs.entity[y] in x and base_pairs.platform[y] in x
for l in corpus.iteritems():
    if base_pairs.apply(lambda pair: pair['entity'] in l and pair['platform'] in l, axis = 1).any():
        res.append(l)

In [11]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

Matplotlib is building the font cache using fc-list. This may take a moment.


In [ ]:
text = ban.entity.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()